In [2]:
from dataset import MyDataset
import numpy as np
from sklearn.svm import SVC
from tqdm import tqdm
import joblib

In [3]:
train_sentence = np.load("./npys/train/sentence_code.npy",allow_pickle=True).tolist()[12000:13000]
y_train = np.load("./npys/train/labels.npy",allow_pickle=True).tolist()[12000:13000]
X_train=[]
for sentence in tqdm (train_sentence):
    total_cnt=0
    temp=list(np.zeros(400000))
    for word in sentence:
        if word>=0:
            temp[word]+=1
            #temp[word]=1
            total_cnt+=1
    for i in range (len(temp)):
        temp[i]/=total_cnt
    X_train.append(temp)
#how to normalize?



100%|██████████| 1000/1000 [01:02<00:00, 16.02it/s]


In [4]:
test_sentence = np.load("./npys/test/sentence_code.npy",allow_pickle=True).tolist()[12000:13000]
y_test = np.load("./npys/train/labels.npy",allow_pickle=True).tolist()[12000:13000]
X_test=[]
for sentence in tqdm (test_sentence):
    total_cnt=0
    temp=list(np.zeros(400000))
    for word in sentence:
        if word>=0:
            temp[word]+=1
            #temp[word]=1
            total_cnt+=1
    for i in range (len(temp)):
        temp[i]/=total_cnt
    X_test.append(temp)

100%|██████████| 1000/1000 [01:06<00:00, 14.96it/s]


In [7]:
X_train[0]


[0.04697986577181208,
 0.05536912751677853,
 0.04194630872483222,
 0.025167785234899327,
 0.018456375838926176,
 0.028523489932885907,
 0.015100671140939598,
 0.016778523489932886,
 0.0016778523489932886,
 0.0,
 0.006711409395973154,
 0.003355704697986577,
 0.016778523489932886,
 0.003355704697986577,
 0.003355704697986577,
 0.013422818791946308,
 0.0,
 0.003355704697986577,
 0.003355704697986577,
 0.016778523489932886,
 0.003355704697986577,
 0.0016778523489932886,
 0.003355704697986577,
 0.006711409395973154,
 0.003355704697986577,
 0.0016778523489932886,
 0.0050335570469798654,
 0.0,
 0.0,
 0.0,
 0.0016778523489932886,
 0.003355704697986577,
 0.0050335570469798654,
 0.006711409395973154,
 0.0016778523489932886,
 0.003355704697986577,
 0.0016778523489932886,
 0.003355704697986577,
 0.0050335570469798654,
 0.010067114093959731,
 0.0016778523489932886,
 0.010067114093959731,
 0.0,
 0.0016778523489932886,
 0.003355704697986577,
 0.0,
 0.0,
 0.0,
 0.0016778523489932886,
 0.0,
 0.0,
 0.0,

In [24]:
try:
    knn = joblib.load('knn.model')
except:
    from sklearn.neighbors import KNeighborsClassifier
    knn=KNeighborsClassifier(n_neighbors=10,metric='cosine',weights="distance").fit(X_train, y_train)
    joblib.dump(knn, 'knn.model')


print("KNN")
print("Accuracy on training set: {:.3f}".format(knn.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(knn.score(X_test, y_test)))

KNN
Accuracy on training set: 1.000
Accuracy on test set: 0.635


使用binary数据 即只看一个单词有没有 不数多少个
KNN
Accuracy on training set: 0.793
Accuracy on test set: 0.510
normal
KNN
Accuracy on training set: 0.795
Accuracy on test set: 0.612

kNN distance n=10
KNN
Accuracy on training set: 1.000
Accuracy on test set: 0.635

In [9]:
#Train decision tree model
try:
    tree = joblib.load('tree.model')
except:
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
    joblib.dump(tree, 'tree.model')

# print("Decision Tree")
# print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
# print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))


Decision Tree
Accuracy on training set: 1.000
Accuracy on test set: 0.667


Binary：
Decision Tree
Accuracy on training set: 1.000
Accuracy on test set: 0.640
normal：
Decision Tree
Accuracy on training set: 1.000
Accuracy on test set: 0.667

In [25]:
try:
    svm = joblib.load('svm.model')
except:
    svm = SVC().fit(X_train, y_train)
    joblib.dump(svm, 'svm.model')

print("Support Vector Machine")
print("Accuracy on training set: {:.3f}".format(svm.score(X_train, y_train)))
print("Accuracy on train set: {:.3f}".format(svm.score(X_test, y_test)))

Support Vector Machine
Accuracy on training set: 0.920
Accuracy on train set: 0.752


Binary：
Support Vector Machine
Accuracy on training set: 0.891
Accuracy on train set: 0.763
Normal:
Support Vector Machine
Accuracy on training set: 0.920
Accuracy on train set: 0.752

In [27]:
tree_predict=tree.predict(X_test)
svm_predict=svm.predict(X_test)
knn_predict=knn.predict(X_test)

In [14]:
#get confusion matrix
#           prediction
#           pos   neg
#tar pos    TP    FN
#get neg    FP    TN
TPlist=[0,0,0]
FNlist=[0,0,0]
FPlist=[0,0,0]
TNlist=[0,0,0]
print(TPlist[0])
def performance(labelArr, predictArr,list_index):  # 样本一定要是数组narray类型 类标签为1，0 # labelArr[i]真实的类别,predictArr[i]预测的类别
    # labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0; TN = 0; FP = 0; FN = 0
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1
        elif labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1
        elif labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1
        elif labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1
    TPlist[list_index]=TP
    FNlist[list_index]=FN
    FPlist[list_index]=FP
    TNlist[list_index]=TN

#生成各模型TP等数据





0


In [29]:
performance(y_test,tree_predict,0)
performance(y_test,knn_predict,1)
performance(y_test,svm_predict,2)

In [36]:
def recall(index):
    TP=TPlist[index]
    FN=FNlist[index]
    TN=TNlist[index]
    FP=FPlist[index]
    recall1=TP/(TP+FN)
    recall2=TN/(FP+TN)
    return recall1,recall2
    
def average_accuracy_hm(index):#index of TPlist
    recall1,recall2=recall(index)
    accuracy=(recall1+recall2)/2
    accuracy_hm=2/(1/recall1+1/recall2)
    return accuracy,accuracy_hm

def precision(index):
    TP=TPlist[index]
    FP=FPlist[index]
    prec=TP/(TP+FP)
    return prec

def F1_feature(index):
    recall1,fuck=recall(index)
    prec=precision(index)
    F1=2*prec*recall1/(prec+recall1)
    return F1


计算各种指标

In [37]:
model_list=["tree","knn","svm"]
for i in range(3):
    print("model:",model_list[i])
    print("recall1 and recall 2",recall(i))
    print("precision",precision(i))
    print("F1 feature",F1_feature(i))
    print("average_accuracy and average_accuracy_hm",average_accuracy_hm(i))
    
    

model: tree
recall1 and recall 2 (0.646, 0.688)
precision 0.6743215031315241
F1 feature 0.6598569969356486
average_accuracy and average_accuracy_hm (0.667, 0.6663388305847077)
model: knn
recall1 and recall 2 (0.656, 0.614)
precision 0.6295585412667947
F1 feature 0.6425073457394711
average_accuracy and average_accuracy_hm (0.635, 0.6343055118110237)
model: svm
recall1 and recall 2 (0.77, 0.734)
precision 0.7432432432432432
F1 feature 0.756385068762279
average_accuracy and average_accuracy_hm (0.752, 0.7515691489361702)


In [38]:
print(TPlist,
FPlist,
TNlist,
FNlist)

[323, 328, 385] [156, 193, 133] [344, 307, 367] [177, 172, 115]


In [ ]:
try:
    tree = joblib.load('tree.model')
except:
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
    joblib.dump(tree, 'tree.model')
tree_predict=tree.predict(X_test)